In [1]:
import os
import sys
import json
from anthropic import Anthropic, AnthropicError

notebook_path = os.getcwd()
project_path = os.path.abspath(os.path.join(notebook_path, '..','..','..'))
print(f"Project path: {project_path}")
# Add the project path to sys.path
sys.path.append(project_path)

Project path: c:\Users\dalej\Documents\_Coding\DragonRegen


### Data Column Type

In [2]:
from src.AIGuardian.Tasks.DataColumnType import DataColumnType
from src.MetaFort.SysLogs.KafkaEngine import KafkaEngine

# Inputs
action_parameters = {
                "table_name": 'members',
                "purpose": "To track all the information about members of the gym.",
                "column_name": 'membership',
                "description": 'The kind of membership the member has.',
            }

test_type_action = DataColumnType(input_params=action_parameters)
test_type_action.db_engine = KafkaEngine.default_builder()
print(test_type_action)

In [3]:
test_type_action.submit_task()

{'task_id': 136058658848764322081540275259036020310,
 'task_name': 'DataColumnType',
 'task_version': '0.0.1',
 'parent_task_id': None,
 'group_task_id': None,
 'description': 'The base Task class. This should be inherited by all Task classes.',
 'sequence_number': 0,
 'created_dt': '2025-05-14T15:06:23.457195',
 'updated_dt': '2025-05-14T15:06:23.457195',
 'input_artifacts': '{"table_name": "members", "purpose": "To track all the information about members of the gym.", "column_name": "membership", "description": "The kind of membership the member has."}'}

In [4]:
await test_type_action.run(user_prompt="")
print(test_type_action.response)

create_params: {'max_tokens': 10000, 'temperature': 0.1, 'stop_sequences': ['</JSON_Template>'], 'model': 'claude-3-7-sonnet-20250219', 'messages': [{'role': 'user', 'content': 'Please take the following <column_information> to choose what type of column a column most likely is.\n<column_information>\nTable Purpose: To track all the information about members of the gym.\nColumn Info: "membership": "The kind of membership the member has."\n</column_information>\n\nChoose one column_type from this list <choices> below, structure as column_type: description of what qualifies a column as that type.\n<choices>\nColBasic: A back stop column for your basic data types with no other identifying attributes.\nStringBasic: This is a basic string column generator that can be used to generate a column of strings with a specified length and format. It is not intended for free text or complex string generation.\nCategorical: a column that represents a category or type, often with a limited set of valu

In [6]:
print(test_type_action.output_params)

{'reason': "The 'membership' column represents the kind of membership a gym member has, which would be a limited set of predefined options (like 'Basic', 'Premium', 'Gold', etc.) rather than free-form text. This is a classic example of a categorical variable with a finite number of possible values.", 'col_type': 'Categorical'}


### Data Column Refiner

In [7]:
from src.AIGuardian.Tasks.DataColumnRefiner import DataColumnRefiner

# Inputs
action_parameters = {
                "table_name": 'members',
                "purpose": "To track all the information about members of the gym.",
                "column_name": 'membership',
                "description": 'The kind of membership the member has.',
            }

test_task = DataColumnRefiner(input_params=action_parameters, parent_task=test_type_action)
# test_task.db_engine = KafkaEngine.default_builder()
print(test_task)

In [8]:
test_task.submit_task()

{'task_id': 104404014328354376757570259899498965908,
 'task_name': 'DataColumnRefiner',
 'task_version': '0.0.1',
 'parent_task_id': 136058658848764322081540275259036020310,
 'group_task_id': 136058658848764322081540275259036020310,
 'description': 'The base Task class. This should be inherited by all Task classes.',
 'sequence_number': 1,
 'created_dt': '2025-05-14T15:06:40.750806',
 'updated_dt': '2025-05-14T15:06:40.750806',
 'input_artifacts': '{"table_name": "members", "purpose": "To track all the information about members of the gym.", "column_name": "membership", "description": "The kind of membership the member has.", "reason": "The \'membership\' column represents the kind of membership a gym member has, which would be a limited set of predefined options (like \'Basic\', \'Premium\', \'Gold\', etc.) rather than free-form text. This is a classic example of a categorical variable with a finite number of possible values.", "col_type": "Categorical"}'}

In [9]:
await test_task.run(user_prompt="")
print(test_task.response)


create_params: {'max_tokens': 10000, 'temperature': 0.1, 'stop_sequences': ['</JSON_Template>'], 'model': 'claude-3-7-sonnet-20250219', 'messages': [{'role': 'user', 'content': 'Please take the following <column_information> to fill in JSON values and only use keys in "metadata" from <JSON_Template>.\n        <column_information>\n        Table Purpose: To track all the information about members of the gym.\n        Column Info: "membership": "The kind of membership the member has."\n        </column_information>\n        Please provide the response in a JSON format like the <JSON_Template> below. If the JSON value has (Optional) in the value you may or may not included.\n        There are examples below at <Examples>.\n        <JSON_Template>\n        {"name": "example_column", "type": "Integer", "nullable": true, "metadata": {"description": "Place the description of the column here. Add detail if current description is vague.", "unique_fl": "Choose True or False", "default_value": "(

In [14]:
print(test_task.response.content[0].text)



  "name": "membership",
  "type": "String",
  "nullable": false,
  "metadata": {
    "description": "The kind of membership the member has. Indicates the membership tier or plan that determines access privileges and pricing for the gym member.",
    "unique_fl": "False",
    "column_values": ["Basic", "Standard", "Premium", "Family", "Student", "Senior", "Corporate"]
  }
}

